## <center> Packages & Connection </center>

In [1]:
import os
import h3 as h3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from pyhive import presto
from keplergl import KeplerGl
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 300)## <center> Local extracted files </center>

## <center> Local extracted files </center>

In [3]:
notebook_path = os.getcwd()
print(notebook_path)

'''/Users/rapido/local-datasets/affluence/raw/'''

/Users/rapido/code-repository/pricing/affluence


In [11]:
raw_order_logs_snapshot = pd.read_csv('/Users/rapido/local-datasets/affluence/raw/raw_order_logs_snapshot_10to16.csv')
raw_order_logs_immutable = pd.read_csv('/Users/rapido/local-datasets/affluence/raw/raw_order_logs_immutable_10to16.csv')
raw_fare_estimates_enriched = pd.read_csv('/Users/rapido/local-datasets/affluence/raw/raw_fare_estimates_enriched_10to16.csv')
raw_iallocator_customer_segments = pd.read_csv('/Users/rapido/local-datasets/affluence/raw/raw_iallocator_customer_segments_10to16.csv')

In [13]:
df_oli_main_data = raw_order_logs_immutable.copy(deep=True)
print(df_oli_main_data.shape)

df_ols_main_data = raw_order_logs_snapshot.copy(deep=True)
print(df_ols_main_data.shape)

df_fee_main_data = raw_fare_estimates_enriched.copy(deep=True)
print(df_fee_main_data.shape)

df_segment_main_data = raw_iallocator_customer_segments.copy(deep=True)
print(df_segment_main_data.shape)

(3536034, 16)
(1149637, 12)
(15117222, 17)
(14739326, 15)


In [14]:
df_fee_main_data.head(2)

,yyyymmdd,city,service_name,quarter_hour,time_bucket,pickup_cluster,pickup_hex_8,drop_cluster,drop_hex_8,customer_id,fare_estimate_id,surge_strategy,surge_percentage,dynamic_surge,dynamic_fare,sub_total,discount_amount
0,20230710,Bangalore,Auto,2015,Evening_Peak,Bellandur,8861892569fffff,Varthur,88618920ebfffff,5e591dcc1275a31e1e5833c9,64ac1af42a70c7ac0b2b56f8,surge_dashboard_rule,10.204082,0.00,10.0,108.0,0.0
1,20230710,Bangalore,Link,1800,Evening_Peak,Basavanagudi,8861892597fffff,Cambridge Layout Ulsoor,8861892ed1fffff,62c843ef0eef7f6e35f11140,64abfc344f76e21a1142b7dc,rain,39.517694,42.77,0.0,151.0,10.0


In [15]:
df_ols_main_data.head(2)

,yyyymmdd,city,service_name,pickup_cluster,pickup_hex_8,drop_cluster,drop_hex_8,estimate_id,order_id,geo_distance,discount,sub_total
0,20230713,Bangalore,Link,Ramaiah College,8860145941fffff,Metro - Sandal Soap Factory,8860145b37fffff,64aff1369c5b35742a429410,64afee9fb82ebd52fc9898da,0.086537,14.0,67.0
1,20230713,Bangalore,Auto,RTO HSR Sec 1,8861892553fffff,Kudlu Gate,886189246bfffff,64aff32f918567f17f3cfa3c,64aff3345332de3f2f5b9e0f,0.013049,0.0,79.0


In [16]:
df_oli_main_data.head(2)

,yyyymmdd,city,service_name,pickup_cluster,pickup_hex_8,drop_cluster,drop_hex_8,estimate_id,gross_orders,requested_orders,cobrm,expiry_mapped,cobra,accepted_orders,ocara,net_orders
0,20230710,Bangalore,Link,HSR Sec 2,886189242dfffff,BTM,88618925ddfffff,64ac0b367912742548bb4557,1,1,0,1,0,0,0,0
1,20230710,Bangalore,Auto,Electronic City,8861892663fffff,Electronic City Phase 2,8861892665fffff,64ac094676627a108e685bd2,1,1,0,0,1,0,0,0


In [17]:
df_segment_main_data.head(2)

,yyyymmdd,customer_id,taxi_high_income,taxi_medium_income,taxi_low_income,link_only_service,auto_only_service,both_service,link_ps,link_nps,auto_ps,auto_nps,fe_intent_stable,fe_intent_increasing,fe_intent_declining
0,20230710,5737c6baddbec2203f7331d9,NaN,NaN,NaN,NaN,NaN,5737c6baddbec2203f7331d9,NaN,5737c6baddbec2203f7331d9,5737c6baddbec2203f7331d9,NaN,5737c6baddbec2203f7331d9,NaN,NaN
1,20230710,5737c6c1ddbec2203f73321e,NaN,5737c6c1ddbec2203f73321e,NaN,5737c6c1ddbec2203f73321e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5737c6c1ddbec2203f73321e


In [18]:
## fee & oli

df_fe_rr_data = pd.merge(df_fee_main_data[df_fee_main_data['service_name'].isin(['Link'])], \
                             df_oli_main_data[df_oli_main_data['service_name'].isin(['Link'])], \
                             how='left', \
                             left_on = ['yyyymmdd', 'city', 'service_name', 'fare_estimate_id'], \
                             right_on = ['yyyymmdd', 'city', 'service_name', 'estimate_id'] \
                            )

In [19]:
df_fe_rr_data[df_fe_rr_data['fare_estimate_id'].isin(['64ace765126ec564e4a789ef','64b201d727b0e89d3cda1123'])]

,yyyymmdd,city,service_name,quarter_hour,time_bucket,pickup_cluster_x,pickup_hex_8_x,drop_cluster_x,drop_hex_8_x,customer_id,fare_estimate_id,surge_strategy,surge_percentage,dynamic_surge,dynamic_fare,sub_total,discount_amount,pickup_cluster_y,pickup_hex_8_y,drop_cluster_y,drop_hex_8_y,estimate_id,gross_orders,requested_orders,cobrm,expiry_mapped,cobra,accepted_orders,ocara,net_orders
1999736,20230711,Bangalore,Link,1045,Morning_Peak,Columbia Asia Whitefield,88618920e7fffff,Whitefield,88618921d3fffff,5fb36b6d7a3a3547d26d5744,64ace765126ec564e4a789ef,mismatch_gradient,21.951220,9.00,0.0,50.0,0.0,Columbia Asia Whitefield,88618920e7fffff,Whitefield,88618921d3fffff,64ace765126ec564e4a789ef,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
6074461,20230715,Bangalore,Link,745,Rest_Morning,Varthur,88618920c1fffff,Kadubeesanahalli,8861892095fffff,625125b349bc1231f17f0c33,64b201d727b0e89d3cda1123,hot_cold_surge,36.843671,24.77,0.0,92.0,0.0,Varthur,88618920c1fffff,Kadubeesanahalli,8861892095fffff,64b201d727b0e89d3cda1123,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [20]:
df_fe_rr_data.columns

Index(['yyyymmdd', 'city', 'service_name', 'quarter_hour', 'time_bucket',
       'pickup_cluster_x', 'pickup_hex_8_x', 'drop_cluster_x', 'drop_hex_8_x',
       'customer_id', 'fare_estimate_id', 'surge_strategy', 'surge_percentage',
       'dynamic_surge', 'dynamic_fare', 'sub_total', 'discount_amount',
       'pickup_cluster_y', 'pickup_hex_8_y', 'drop_cluster_y', 'drop_hex_8_y',
       'estimate_id', 'gross_orders', 'requested_orders', 'cobrm',
       'expiry_mapped', 'cobra', 'accepted_orders', 'ocara', 'net_orders'],
      dtype='object')

In [21]:
df_fe_rr_data.rename(columns = {'pickup_cluster_x' : 'pickup_cluster', \
                                'pickup_hex_8_x' : 'pickup_hex_8', \
                                'drop_cluster_x' : 'drop_cluster', \
                                'drop_hex_8_x' : 'drop_hex_8', \
                                'discount_amount' : 'discount_amount_fe' \
                               }, 
                    inplace = True
                    )

In [22]:
df_fe_rr_data = df_fe_rr_data[['yyyymmdd', 'city', 'service_name', 'quarter_hour', 'time_bucket', \
                               'pickup_cluster', 'pickup_hex_8', 'drop_cluster', 'drop_hex_8', \
                               'customer_id', 'fare_estimate_id', 'surge_strategy', 'surge_percentage', \
                               'dynamic_surge', 'dynamic_fare', 'sub_total', 'discount_amount_fe', \
                               'estimate_id', 'gross_orders', 'requested_orders', 'cobrm', \
                               'expiry_mapped', 'cobra', 'accepted_orders', 'ocara', 'net_orders']]

In [23]:
df_fe_rr_data.head(2)

,yyyymmdd,city,service_name,quarter_hour,time_bucket,pickup_cluster,pickup_hex_8,drop_cluster,drop_hex_8,customer_id,fare_estimate_id,surge_strategy,surge_percentage,dynamic_surge,dynamic_fare,sub_total,discount_amount_fe,estimate_id,gross_orders,requested_orders,cobrm,expiry_mapped,cobra,accepted_orders,ocara,net_orders
0,20230710,Bangalore,Link,1800,Evening_Peak,Basavanagudi,8861892597fffff,Cambridge Layout Ulsoor,8861892ed1fffff,62c843ef0eef7f6e35f11140,64abfc344f76e21a1142b7dc,rain,39.517694,42.77,0.0,151.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230710,Bangalore,Link,1015,Morning_Peak,Basaveshwar Nagar,886014586bfffff,Yeshwanthpur,886014595bfffff,6399734853245a499edb0326,64ab8e968d1303f14c3e0cf9,no_surge,0.000000,0.00,0.0,47.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
## fe_rr & ols

df_fe_net_data = pd.merge(df_fe_rr_data, \
                             df_ols_main_data[df_ols_main_data['service_name'].isin(['Link'])], \
                             how='left', \
                             left_on = ['yyyymmdd', 'city', 'service_name', 'fare_estimate_id'], \
                             right_on = ['yyyymmdd', 'city', 'service_name', 'estimate_id'] \
                            )

In [25]:
df_fe_net_data.columns

Index(['yyyymmdd', 'city', 'service_name', 'quarter_hour', 'time_bucket',
       'pickup_cluster_x', 'pickup_hex_8_x', 'drop_cluster_x', 'drop_hex_8_x',
       'customer_id', 'fare_estimate_id', 'surge_strategy', 'surge_percentage',
       'dynamic_surge', 'dynamic_fare', 'sub_total_x', 'discount_amount_fe',
       'estimate_id_x', 'gross_orders', 'requested_orders', 'cobrm',
       'expiry_mapped', 'cobra', 'accepted_orders', 'ocara', 'net_orders',
       'pickup_cluster_y', 'pickup_hex_8_y', 'drop_cluster_y', 'drop_hex_8_y',
       'estimate_id_y', 'order_id', 'geo_distance', 'discount', 'sub_total_y'],
      dtype='object')

In [26]:
df_fe_net_data.rename(columns = {'pickup_cluster_x' : 'pickup_cluster', \
                                'pickup_hex_8_x' : 'pickup_hex_8', \
                                'drop_cluster_x' : 'drop_cluster', \
                                'drop_hex_8_x' : 'drop_hex_8', \
                                'estimate_id_x' : 'estimate_id', \
                                'sub_total_x' : 'sub_total_fe', \
                                'sub_total_y' : 'sub_total_gross', \
                                'discount_amount' : 'discount_amount_fe', \
                                'discount' : 'discount_gross'
                               }, 
                    inplace = True
                    )

In [27]:
# New columns
df_fe_net_data['surge_strategy_applied'] = np.where( \
                                                     df_fe_net_data['surge_strategy'] \
                                                     .isin(['rain', 'mismatch_generic', 'hot_cold_surge', \
                                                            'mismatch_gradient', 'circuit_broken', \
                                                            'surge_dashboard_rule']),    
                                                     df_fe_net_data['fare_estimate_id'],
                                                     None)
df_fe_net_data['geo_distance_1'] = df_fe_net_data['geo_distance']

In [28]:
df_refined_data =  df_fe_net_data.groupby(['pickup_cluster', 'pickup_hex_8']).agg({'fare_estimate_id' : 'nunique', \
                                                                 'gross_orders' : 'sum', \
                                                                 'order_id' : 'nunique', \
                                                                 'surge_strategy_applied' : 'nunique', \
                                                                 'surge_percentage' : 'mean', \
                                                                 'discount_amount_fe' : 'sum', \
                                                                 'discount_gross' : 'sum', \
                                                                 'sub_total_fe' : 'sum', \
                                                                 'customer_id' : 'nunique', \
                                                                 'geo_distance' : 'mean', \
                                                                 'geo_distance_1' : 'max' \
                                                                }).reset_index()
df_refined_data.head(5)
#df_refined_data.columns = df_refined_data.columns.droplevel(1)
df_refined_data.rename(columns = {'fare_estimate_id' : 'fe_count', \
                                  'order_id' : 'net_orders',
                                  'surge_strategy_applied' : 'surged_fe', \
                                  'customer_id' : 'fe_customer_count', \
                                  'geo_distance' : 'mean_geo_distance', \
                                  'geo_distance_1' : 'max_geo_distance' \
                                 }, inplace = True)

In [29]:
df_refined_data.head(5)

,pickup_cluster,pickup_hex_8,fe_count,gross_orders,net_orders,surged_fe,surge_percentage,discount_amount_fe,discount_gross,sub_total_fe,fe_customer_count,mean_geo_distance,max_geo_distance
0,AECS Layout Brookefield,88618920a3fffff,40894,9296.0,4076,17523,12.150350,84670.0,27540.0,4382022.0,10354,0.033276,8.421555
1,Adugodi,8861892581fffff,16022,2800.0,1016,4570,6.840293,35883.0,7374.0,1498201.0,3934,0.038787,3.909074
2,Adugodi,886189258bfffff,19192,3468.0,1049,6057,8.488439,46275.0,6850.0,1840822.0,4230,0.050087,1.307781
3,Adugodi Traffic Station,886189258dfffff,25303,4315.0,1498,4062,3.597522,52512.0,3191.0,2135209.0,5771,0.035588,0.886427
4,Agara Lake,8861892425fffff,18454,3483.0,1375,6368,8.956572,38155.0,11506.0,1724176.0,4783,0.028703,1.060272


In [30]:
# new columns 
df_refined_data['fe2rr'] = round(df_refined_data['gross_orders']*100.00/df_refined_data['fe_count'], 2)
df_refined_data['fe2net'] = round(df_refined_data['net_orders']*100.00/df_refined_data['fe_count'], 2)
df_refined_data['g2n'] = round(df_refined_data['net_orders']*100.00/df_refined_data['gross_orders'], 2)

In [31]:
df_refined_data.head(5)

,pickup_cluster,pickup_hex_8,fe_count,gross_orders,net_orders,surged_fe,surge_percentage,discount_amount_fe,discount_gross,sub_total_fe,fe_customer_count,mean_geo_distance,max_geo_distance,fe2rr,fe2net,g2n
0,AECS Layout Brookefield,88618920a3fffff,40894,9296.0,4076,17523,12.150350,84670.0,27540.0,4382022.0,10354,0.033276,8.421555,22.73,9.97,43.85
1,Adugodi,8861892581fffff,16022,2800.0,1016,4570,6.840293,35883.0,7374.0,1498201.0,3934,0.038787,3.909074,17.48,6.34,36.29
2,Adugodi,886189258bfffff,19192,3468.0,1049,6057,8.488439,46275.0,6850.0,1840822.0,4230,0.050087,1.307781,18.07,5.47,30.25
3,Adugodi Traffic Station,886189258dfffff,25303,4315.0,1498,4062,3.597522,52512.0,3191.0,2135209.0,5771,0.035588,0.886427,17.05,5.92,34.72
4,Agara Lake,8861892425fffff,18454,3483.0,1375,6368,8.956572,38155.0,11506.0,1724176.0,4783,0.028703,1.060272,18.87,7.45,39.48


In [32]:
## low and high aff tag
raw_list_hex_8 = pd.read_csv('list_hex_8.csv')
df_list_hex_8 = raw_list_hex_8.copy(deep=True)
print(df_list_hex_8.shape)
df_list_hex_8.head(2)

(2684, 3)


,pickup_hex_8,income_signal,density_signal
0,88618920a3fffff,High Income,High FE
1,8861892581fffff,High Income,High FE


In [33]:
df_refined_data_with_tag = pd.merge (df_refined_data, \
                                     df_list_hex_8, \
                                     how = 'left', \
                                     left_on = ['pickup_hex_8'], \
                                     right_on = ['pickup_hex_8']
                                    )
df_refined_data_with_tag[df_refined_data_with_tag['net_orders'] > 0]

,pickup_cluster,pickup_hex_8,fe_count,gross_orders,net_orders,surged_fe,surge_percentage,discount_amount_fe,discount_gross,sub_total_fe,fe_customer_count,mean_geo_distance,max_geo_distance,fe2rr,fe2net,g2n,income_signal,density_signal
0,AECS Layout Brookefield,88618920a3fffff,40894,9296.0,4076,17523,12.150350,84670.0,27540.0,4382022.0,10354,0.033276,8.421555,22.73,9.97,43.85,High Income,High FE
1,Adugodi,8861892581fffff,16022,2800.0,1016,4570,6.840293,35883.0,7374.0,1498201.0,3934,0.038787,3.909074,17.48,6.34,36.29,High Income,High FE
2,Adugodi,886189258bfffff,19192,3468.0,1049,6057,8.488439,46275.0,6850.0,1840822.0,4230,0.050087,1.307781,18.07,5.47,30.25,High Income,High FE
3,Adugodi Traffic Station,886189258dfffff,25303,4315.0,1498,4062,3.597522,52512.0,3191.0,2135209.0,5771,0.035588,0.886427,17.05,5.92,34.72,High Income,High FE
4,Agara Lake,8861892425fffff,18454,3483.0,1375,6368,8.956572,38155.0,11506.0,1724176.0,4783,0.028703,1.060272,18.87,7.45,39.48,High Income,High FE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2679,Yeshwanthpur,8860145955fffff,2669,399.0,139,734,5.927174,6138.0,1356.0,356018.0,1050,0.087592,2.277133,14.95,5.21,34.84,High Income,Low FE
2680,Yeshwanthpur,8860145957fffff,7538,1531.0,595,2252,5.762536,14145.0,6440.0,977557.0,3033,0.101380,6.808805,20.31,7.89,38.86,High Income,High FE
2681,Yeshwanthpur,8860145959fffff,2799,541.0,234,646,4.823473,6765.0,2650.0,308677.0,904,0.037892,0.867591,19.33,8.36,43.25,High Income,Low FE
2682,Yeshwanthpur,886014595bfffff,2514,485.0,221,687,5.156348,5860.0,2307.0,288735.0,857,0.034007,0.383501,19.29,8.79,45.57,High Income,Low FE


In [34]:
df_refined_data_with_tag.groupby(['income_signal', 'density_signal']).agg({'pickup_hex_8' : 'count', \
                                                                           'fe_count' : 'sum', \
                                                                           'gross_orders' : 'sum', \
                                                                           'net_orders' : 'sum', \
                                                                           'surged_fe' : 'sum' \
                                                                          })

pickup_hex_8  fe_count  gross_orders  \
income_signal density_signal                                         
High Income   High FE                  490   6320789     1162054.0   
              Low FE                   684    474520       89756.0   
Low Income    High FE                   47    390148       64620.0   
              Low FE                  1463    351639       61600.0   

                              net_orders  surged_fe  
income_signal density_signal                         
High Income   High FE             456850    2744102  
              Low FE               35846     171573  
Low Income    High FE              25012     158135  
              Low FE               22672     128804

In [35]:
df_refined_data_with_tag_v1 = df_refined_data_with_tag[df_refined_data_with_tag['density_signal'] \
                                                       .isin(['High FE']) \
                                                      &
                                                      df_refined_data_with_tag['income_signal'] \
                                                       .isin(['High Income']) \
                                                      ]
df_refined_data_with_tag_v2 = df_refined_data_with_tag[df_refined_data_with_tag['density_signal'] \
                                                       .isin(['High FE']) \
                                                      &
                                                      df_refined_data_with_tag['income_signal'] \
                                                       .isin(['Low Income']) 
                                                        &
                                                       df_refined_data_with_tag['gross_orders'] > 0
                                                      ]

In [36]:
df_refined_data_with_tag_v2.pickup_hex_8.nunique()

45

In [37]:
df_refined_data_with_tag_v2.to_csv('45_hex_reference.csv', index = False)

In [38]:
df_refined_data_with_tag_v2.shape

(45, 18)

In [39]:
## Group TC

In [40]:
df_refined_data_with_tag_v2.columns

Index(['pickup_cluster', 'pickup_hex_8', 'fe_count', 'gross_orders',
       'net_orders', 'surged_fe', 'surge_percentage', 'discount_amount_fe',
       'discount_gross', 'sub_total_fe', 'fe_customer_count',
       'mean_geo_distance', 'max_geo_distance', 'fe2rr', 'fe2net', 'g2n',
       'income_signal', 'density_signal'],
      dtype='object')

In [41]:
test_control_grouper= ['fe2rr'] # [ 'net_orders', 'g2n', 'fe2rr', 'fe2net', 'mean_geo_distance']

df_refined_data_group_tc = df_refined_data_with_tag_v2.rename(columns={'pickup_hex_8':'hex_8'})

In [42]:
df_refined_data_with_tag_v2.g2n.describe()

count    45.000000
mean     40.356000
std       4.905149
min      29.450000
25%      37.010000
50%      40.400000
75%      44.040000
max      49.250000
Name: g2n, dtype: float64

In [43]:
df_refined_data_with_tag_v2.fe2rr.describe()

count    45.000000
mean     18.222889
std       3.153569
min       9.490000
25%      16.370000
50%      17.960000
75%      20.530000
max      23.010000
Name: fe2rr, dtype: float64

In [44]:
#group_hex_severe_df = group_hex_severe.copy()

test_control_grouper=['fe2rr']


In [45]:
test_group_raw_v1 = df_refined_data_with_tag_v2.groupby(test_control_grouper, group_keys=False).apply(
            lambda x: x.sample(frac=0.51, random_state=42))

In [46]:
test_group_raw_v1.shape

(45, 18)

In [51]:
df_refined_data_with_tag_v2.sort_values(['net_orders'], ascending = False)

,pickup_cluster,pickup_hex_8,fe_count,gross_orders,net_orders,surged_fe,surge_percentage,discount_amount_fe,discount_gross,sub_total_fe,fe_customer_count,mean_geo_distance,max_geo_distance,fe2rr,fe2net,g2n,income_signal,density_signal
1669,Majestic,8860145b55fffff,73331,10560.0,3562,34511,12.737401,154531.0,27834.0,10715963.0,25922,0.084987,13.159235,14.40,4.86,33.73,Low Income,High FE
2268,ShivajiNagar - Bangalore,8861892e93fffff,28446,4015.0,1464,14735,14.827950,66994.0,11887.0,3248268.0,8210,0.045451,0.935024,14.11,5.15,36.46,Low Income,High FE
2683,Yeshwanthpur - Mathikere,886014594bfffff,11222,2501.0,1228,2145,3.664665,25445.0,12323.0,1297564.0,3833,0.055065,2.787774,22.29,10.94,49.10,Low Income,High FE
1732,Metro - Yeshwantpur,886014595dfffff,20950,2809.0,956,3938,3.587542,53180.0,5986.0,3245776.0,7964,0.118246,10.634049,13.41,4.56,34.03,Low Income,High FE
2020,Parappana Agrahara,886189244dfffff,10326,2072.0,911,3736,7.253473,22059.0,9339.0,1121522.0,3040,0.042807,3.189051,20.07,8.82,43.97,Low Income,High FE
1667,Majestic,8860145b51fffff,11177,2001.0,812,6657,16.275277,25344.0,6094.0,1312984.0,3801,0.062213,3.882212,17.90,7.26,40.58,Low Income,High FE
12,Anepalya,88618925e3fffff,9507,2114.0,754,1440,3.565109,14476.0,6514.0,898555.0,2258,0.036760,0.635915,22.24,7.93,35.67,Low Income,High FE
236,Banaswadi,8861892e85fffff,15998,1934.0,679,7524,11.831357,43961.0,8497.0,2511374.0,5284,0.087548,9.971067,12.09,4.24,35.11,Low Income,High FE
2036,Peenya,8860145913fffff,7890,1620.0,676,3119,7.486815,15839.0,7001.0,944582.0,3089,0.058283,2.230040,20.53,8.57,41.73,Low Income,High FE
1671,Majestic,8860145b59fffff,15804,2265.0,667,10227,18.393741,34730.0,5005.0,1814884.0,5165,0.060632,3.773803,14.33,4.22,29.45,Low Income,High FE
